<a href="https://colab.research.google.com/github/sachincs3108/Hands-on_project/blob/master/Copy_of_regression_using_ML_Pipeline_with_PySpark_in_Databricks_on_US_cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pyspark
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
spark_new = SparkSession.builder.master("local[2]").getOrCreate()

In [ ]:
data = spark_new.read.option("header" , True).option("inferSchema",True).csv("/FileStore/tables/USA_cars_datasets.csv")

In [ ]:
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- brand: string (nullable = true)
 |-- model: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- title_status: string (nullable = true)
 |-- mileage: double (nullable = true)
 |-- color: string (nullable = true)
 |-- vin: string (nullable = true)
 |-- lot: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- condition: string (nullable = true)



In [ ]:
data.show()

+---+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+
|_c0|price|    brand|  model|year| title_status| mileage| color|                vin|      lot|         state|country|    condition|
+---+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+
|  0| 6300|   toyota|cruiser|2008|clean vehicle|274117.0| black|  jtezu11f88k007763|159348797|    new jersey|    usa| 10 days left|
|  1| 2899|     ford|     se|2011|clean vehicle|190552.0|silver|  2fmdk3gc4bbb02217|166951262|     tennessee|    usa|  6 days left|
|  2| 5350|    dodge|    mpv|2018|clean vehicle| 39590.0|silver|  3c4pdcgg5jt346413|167655728|       georgia|    usa|  2 days left|
|  3|25000|     ford|   door|2014|clean vehicle| 64146.0|  blue|  1ftfw1et4efc23745|167753855|      virginia|    usa|22 hours left|
|  4|27700|chevrolet|   1500|2018|clean vehicle|  6654.0|   red|  3gcpcrec2j

In [ ]:
df_panda = data.toPandas()

In [ ]:
df_panda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _c0           2499 non-null   int32  
 1   price         2499 non-null   int32  
 2   brand         2499 non-null   object 
 3   model         2499 non-null   object 
 4   year          2499 non-null   int32  
 5   title_status  2499 non-null   object 
 6   mileage       2499 non-null   float64
 7   color         2499 non-null   object 
 8   vin           2499 non-null   object 
 9   lot           2499 non-null   int32  
 10  state         2499 non-null   object 
 11  country       2499 non-null   object 
 12  condition     2499 non-null   object 
dtypes: float64(1), int32(4), object(8)
memory usage: 214.9+ KB


In [ ]:
df_panda.dtypes

Out[45]: _c0               int32
price             int32
brand            object
model            object
year              int32
title_status     object
mileage         float64
color            object
vin              object
lot               int32
state            object
country          object
condition        object
dtype: object

In [ ]:
df_panda.isna().sum()

Out[46]: _c0             0
price           0
brand           0
model           0
year            0
title_status    0
mileage         0
color           0
vin             0
lot             0
state           0
country         0
condition       0
dtype: int64

In [ ]:
df_panda.describe()

,_c0,price,year,mileage,lot
count,2499.000000,2499.000000,2499.000000,2.499000e+03,2.499000e+03
mean,1249.000000,18767.671469,2016.714286,5.229869e+04,1.676914e+08
std,721.543484,12116.094936,3.442656,5.970552e+04,2.038772e+05
min,0.000000,0.000000,1973.000000,0.000000e+00,1.593488e+08
25%,624.500000,10200.000000,2016.000000,2.146650e+04,1.676253e+08
50%,1249.000000,16900.000000,2018.000000,3.536500e+04,1.677451e+08
75%,1873.500000,25555.500000,2019.000000,6.347250e+04,1.677798e+08
max,2498.000000,84900.000000,2020.000000,1.017936e+06,1.678055e+08


In [ ]:
df_panda.columns

Out[48]: Index(['_c0', 'price', 'brand', 'model', 'year', 'title_status', 'mileage',
       'color', 'vin', 'lot', 'state', 'country', 'condition'],
      dtype='object')

In [ ]:
# check without encoding data i.e with non numeric columns

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['_c0', 'price', 'brand', 'model', 'year', 'title_status', 'mileage',
       'color', 'vin', 'lot', 'state', 'country', 'condition'], outputCol = 'feature_vector')

In [ ]:
# data_new = assembler.transform(data)  (gives error so  we use numeric data)

In [ ]:
assembler = VectorAssembler(inputCols = [ 'year',  'mileage','lot'], outputCol = 'feature_vector')

In [ ]:
data_new = assembler.transform(data)

In [ ]:
data_new.show()

+---+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+--------------------+
|_c0|price|    brand|  model|year| title_status| mileage| color|                vin|      lot|         state|country|    condition|      feature_vector|
+---+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+--------------------+
|  0| 6300|   toyota|cruiser|2008|clean vehicle|274117.0| black|  jtezu11f88k007763|159348797|    new jersey|    usa| 10 days left|[2008.0,274117.0,...|
|  1| 2899|     ford|     se|2011|clean vehicle|190552.0|silver|  2fmdk3gc4bbb02217|166951262|     tennessee|    usa|  6 days left|[2011.0,190552.0,...|
|  2| 5350|    dodge|    mpv|2018|clean vehicle| 39590.0|silver|  3c4pdcgg5jt346413|167655728|       georgia|    usa|  2 days left|[2018.0,39590.0,1...|
|  3|25000|     ford|   door|2014|clean vehicle| 64146.0|  blue|  1ftfw1et4efc2374

In [ ]:
data_new.select('feature_vector').show()

+--------------------+
|      feature_vector|
+--------------------+
|[2008.0,274117.0,...|
|[2011.0,190552.0,...|
|[2018.0,39590.0,1...|
|[2014.0,64146.0,1...|
|[2018.0,6654.0,1....|
|[2018.0,45561.0,1...|
|[2010.0,149050.0,...|
|[2017.0,23525.0,1...|
|[2018.0,9371.0,1....|
|[2017.0,63418.0,1...|
|[2009.0,107856.0,...|
|[2017.0,39650.0,1...|
|[2018.0,22909.0,1...|
|[2017.0,138650.0,...|
|[2013.0,100757.0,...|
|[2017.0,25747.0,1...|
|[2018.0,46194.0,1...|
|[2018.0,20002.0,1...|
|[2017.0,35714.0,1...|
|[2013.0,54380.0,1...|
+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.ml.regression import RandomForestRegressor
rf_mod = RandomForestRegressor(featuresCol = 'feature_vector', labelCol = 'price')

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
pm = ParamGridBuilder().addGrid(rf_mod.numTrees,[100,1500]).build()

In [ ]:
# cross validation
from pyspark.ml.tuning import CrossValidator
cf = CrossValidator(estimator=rf_mod,estimatorParamMaps= pm,evaluator=RegressionEvaluator(labelCol="price"),numFolds=3)

In [ ]:
# splitting data
tr_data, test_data = data_new.randomSplit([0.75,0.25],seed =100)

In [ ]:
tr_data.select('feature_vector').show()

+--------------------+
|      feature_vector|
+--------------------+
|[2008.0,274117.0,...|
|[2011.0,190552.0,...|
|[2018.0,39590.0,1...|
|[2018.0,6654.0,1....|
|[2018.0,45561.0,1...|
|[2010.0,149050.0,...|
|[2017.0,23525.0,1...|
|[2018.0,9371.0,1....|
|[2017.0,63418.0,1...|
|[2018.0,22909.0,1...|
|[2017.0,138650.0,...|
|[2017.0,25747.0,1...|
|[2018.0,46194.0,1...|
|[2018.0,20002.0,1...|
|[2017.0,35714.0,1...|
|[2013.0,54380.0,1...|
|[2018.0,38823.0,1...|
|[2015.0,61578.0,1...|
|[2017.0,46010.0,1...|
|[2013.0,105510.0,...|
+--------------------+
only showing top 20 rows



In [ ]:
# training
cvmod =cf.fit(tr_data)

In [ ]:
bestModel = cvmod.bestModel

In [ ]:
bestModel

Out[92]: RandomForestRegressionModel: uid=RandomForestRegressor_ad7af5b2699b, numTrees=1500, numFeatures=3

In [ ]:
pred = cvmod.transform(test_data)

In [ ]:
pred.show()

+---+-----+-------------+-----------+----+-------------+--------+--------+-------------------+---------+--------------+-------+-------------+--------------------+------------------+
|_c0|price|        brand|      model|year| title_status| mileage|   color|                vin|      lot|         state|country|    condition|      feature_vector|        prediction|
+---+-----+-------------+-----------+----+-------------+--------+--------+-------------------+---------+--------------+-------+-------------+--------------------+------------------+
|  3|25000|         ford|       door|2014|clean vehicle| 64146.0|    blue|  1ftfw1et4efc23745|167753855|      virginia|    usa|22 hours left|[2014.0,64146.0,1...| 16487.32455223293|
| 10|10400|        dodge|      coupe|2009|clean vehicle|107856.0|  orange|  2b3lj54t49h509675|167753874|       georgia|    usa|22 hours left|[2009.0,107856.0,...| 7866.548908001573|
| 11|12920|          gmc|        mpv|2017|clean vehicle| 39650.0|   white|  1gks2bkc6hr136

In [ ]:
pred_mod = pred.select('price','prediction')

In [ ]:
pred_mod.show()

+-----+------------------+
|price|        prediction|
+-----+------------------+
|25000| 16487.32455223293|
|10400| 7866.548908001573|
|12920|11055.986247847994|
|20700| 9978.177624722264|
|12520|12871.046669579215|
|11900|13654.261506146186|
|13000| 8092.313044244285|
|13000| 15776.98636342329|
|11900|13654.261506146186|
|19200|27451.185641050106|
| 7320| 14934.87209316283|
|12710|12871.046669579215|
|16600| 14533.55297887039|
| 5580|14133.537496562158|
|27000|22570.061519269282|
|23000| 25070.14459340466|
|18500| 23603.59587087614|
|12710|11240.802990697883|
|21100| 20285.94192635536|
|19300|23676.967630620475|
+-----+------------------+
only showing top 20 rows



In [ ]:
pred = pred.withColumnRenamed('price','label')

In [ ]:
pred.show()

+---+-----+-------------+-----------+----+-------------+--------+--------+-------------------+---------+--------------+-------+-------------+--------------------+------------------+
|_c0|label|        brand|      model|year| title_status| mileage|   color|                vin|      lot|         state|country|    condition|      feature_vector|        prediction|
+---+-----+-------------+-----------+----+-------------+--------+--------+-------------------+---------+--------------+-------+-------------+--------------------+------------------+
|  3|25000|         ford|       door|2014|clean vehicle| 64146.0|    blue|  1ftfw1et4efc23745|167753855|      virginia|    usa|22 hours left|[2014.0,64146.0,1...| 16487.32455223293|
| 10|10400|        dodge|      coupe|2009|clean vehicle|107856.0|  orange|  2b3lj54t49h509675|167753874|       georgia|    usa|22 hours left|[2009.0,107856.0,...| 7866.548908001573|
| 11|12920|          gmc|        mpv|2017|clean vehicle| 39650.0|   white|  1gks2bkc6hr136

In [ ]:
evaluator = RegressionEvaluator()
eval_score = evaluator.evaluate(pred,{evaluator.metricName:'r2'})

In [ ]:
eval_score

Out[90]: 0.36146040916119515

In [ ]:
# Encoding the data 
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [ ]:
string_index = StringIndexer(inputCols = ['brand', 'model', 'color', 'state', ], outputCols = ['brand_idx', 'model_idx','color_idx','state_idx'])


In [ ]:
data1 = string_index.fit(data).transform(data)

In [ ]:
data1.show()

+---+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+---------+---------+---------+---------+
|_c0|price|    brand|  model|year| title_status| mileage| color|                vin|      lot|         state|country|    condition|brand_idx|model_idx|color_idx|state_idx|
+---+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+---------+---------+---------+---------+
|  0| 6300|   toyota|cruiser|2008|clean vehicle|274117.0| black|  jtezu11f88k007763|159348797|    new jersey|    usa| 10 days left|     27.0|     89.0|      1.0|     10.0|
|  1| 2899|     ford|     se|2011|clean vehicle|190552.0|silver|  2fmdk3gc4bbb02217|166951262|     tennessee|    usa|  6 days left|      0.0|    113.0|      3.0|     21.0|
|  2| 5350|    dodge|    mpv|2018|clean vehicle| 39590.0|silver|  3c4pdcgg5jt346413|167655728|       georgia|    usa|  2 days left|      1.0

In [ ]:
ohe_mod = OneHotEncoder(inputCols = ['brand_idx', 'model_idx','color_idx','state_idx'], outputCols = ['brand_ohe', 'model_ohe','color_ohe','state_ohe'])

In [ ]:
data1 = ohe_mod.fit(data1).transform(data1)

In [ ]:
data1.show()

+---+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+---------+---------+---------+---------+---------------+-----------------+--------------+---------------+
|_c0|price|    brand|  model|year| title_status| mileage| color|                vin|      lot|         state|country|    condition|brand_idx|model_idx|color_idx|state_idx|      brand_ohe|        model_ohe|     color_ohe|      state_ohe|
+---+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+---------+---------+---------+---------+---------------+-----------------+--------------+---------------+
|  0| 6300|   toyota|cruiser|2008|clean vehicle|274117.0| black|  jtezu11f88k007763|159348797|    new jersey|    usa| 10 days left|     27.0|     89.0|      1.0|     10.0|     (27,[],[])| (126,[89],[1.0])|(48,[1],[1.0])|(43,[10],[1.0])|
|  1| 2899|     ford|     se|2011|clean vehicle|1905

In [ ]:
assembler = VectorAssembler(inputCols = [ 'brand_idx', 'model_idx','color_idx','state_idx','year',  'mileage','lot'], outputCol = 'feature_vector')

In [ ]:
data_new = assembler.transform(data1)

In [ ]:
data_new.select('feature_vector').show()

+--------------------+
|      feature_vector|
+--------------------+
|[27.0,89.0,1.0,10...|
|[0.0,113.0,3.0,21...|
|[1.0,4.0,3.0,15.0...|
|[0.0,0.0,5.0,9.0,...|
|[3.0,15.0,4.0,1.0...|
|[1.0,4.0,0.0,2.0,...|
|[3.0,54.0,1.0,15....|
|[4.0,0.0,2.0,3.0,...|
|[3.0,36.0,3.0,1.0...|
|[0.0,4.0,1.0,2.0,...|
|[1.0,49.0,8.0,15....|
|[4.0,4.0,0.0,3.0,...|
|[3.0,15.0,1.0,21....|
|[6.0,22.0,2.0,2.0...|
|[0.0,0.0,1.0,9.0,...|
|[4.0,0.0,0.0,3.0,...|
|[10.0,77.0,5.0,5....|
|[9.0,65.0,4.0,21....|
|[0.0,4.0,0.0,2.0,...|
|[0.0,0.0,2.0,9.0,...|
+--------------------+
only showing top 20 rows

